---
title: IDs from ARTHEMIS
order: 13
---

See following notebooks for details:

- [Magnetic field data](./mag.ipynb)
- [State data](./state.ipynb)

## Background

ARTEMIS spacecrafts will be exposed in the solar wind at 1 AU during its orbits around the Moon. So it's very interesting to look into its data.

- For time inteval for THEMIS-B in solar wind, see [Link](https://omniweb.gsfc.nasa.gov/ftpbrowser/themis_b_sw.txt)
- For time inteval for THEMIS-C in solar wind, see [Link](https://omniweb.gsfc.nasa.gov/ftpbrowser/themis_c_sw.txt)


In [1]:
import polars as pl
from space_analysis.utils.speasy import Variables
from discontinuitypy.datasets import IDsDataset
from discontinuitypy.utils.basic import resample
from beforerr.polars import pl_norm

from datetime import timedelta
from sunpy.time import TimeRange

from space_analysis.plasma.formulary import df_thermal_spd2temp

from tqdm import tqdm

25-Feb-24 21:20:54: UserWarning: Traceback (most recent call last):
  File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/__init__.py", line 85, in <module>
    from . import skintegrate
  File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/skintegrate.py", line 20, in <module>
    from sklearn.base import BaseEstimator
ModuleNotFoundError: No module named 'sklearn'


25-Feb-24 21:20:54: UserWarning: pdpipe: Scikit-learn or skutil import failed. Scikit-learn-dependent pipeline stages will not be loaded.

25-Feb-24 21:20:54: UserWarning: Traceback (most recent call last):
  File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/__init__.py", line 105, in <module>
    from . import nltk_stages
  File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/nltk_stages.py", line 19, in <module>
    import nltk
ModuleNotFoundError: No module named 'nltk'


25-Feb-2

In [2]:
timerange = ["2011-08-25", "2016-06-30"]
timerange = ["2011-08-25", "2011-09-01"]
mission = "THB"
ts = timedelta(seconds=1)
tau = timedelta(seconds=60)

provider = 'archive/local'
mag_dataset = "THB_L2_FGM"
mag_parameters = ["thb_fgl_gse"]
plasma_dataset = 'THB_L2_MOM'
plasma_parameters = ["thb_peim_densityQ", "thb_peim_velocity_gseQ"]

fmt = 'arrow'
fname = f"../../../data/05_reporting/events.{mission}.ts_{ts.total_seconds():.2f}s_tau_60s.{fmt}"

In [8]:
timerange = ["2011-08-25", "2016-06-30"]
timerange = ["2011-08-25", "2011-09-01"]

mission = "Wind"
ts = timedelta(seconds=1)
tau = timedelta(seconds=60)

provider = 'archive/local'
mag_dataset = "WI_H4-RTN_MFI"
mag_parameters = ["BRTN"]
plasma_dataset = "WI_K0_SWE"
plasma_parameters= ["Np", "V_GSM", "THERMAL_SPD"]

fmt = 'arrow'
fname = f"../../data/05_reporting/events.{mission}.ts_{ts.total_seconds():.2f}s_tau_60s.{fmt}"

In [9]:
mag_vars = Variables(
    provider = provider,
    dataset=mag_dataset,
    parameters=mag_parameters,
    timerange=timerange,
).retrieve_data()

In [10]:
mag_vars.time_resolutions

statistic,value
str,str
"""count""","""6053744"""
"""null_count""","""1"""
"""mean""","""0:00:00.092974…"
"""min""","""0:00:00.091000…"
"""25%""","""0:00:00.092000…"
"""50%""","""0:00:00.092000…"
"""75%""","""0:00:00.092000…"
"""max""","""0:04:36.092000…"


In [7]:
plasma_vars = Variables(
    provider = provider,
    dataset=plasma_dataset,
    parameters=plasma_parameters,
    timerange=timerange,
).retrieve_data()

In [9]:
def get_and_process_data(
    mag_dataset, mag_parameters, plasma_dataset, plasma_parameters, timerange, tau, ts, provider
):
    # define variables
    mag_vars = Variables(
        provider = provider,
        dataset=mag_dataset,
        parameters=mag_parameters,
        timerange=timerange,
    ).retrieve_data()

    plasma_vars = Variables(
        provider = provider,
        dataset=plasma_dataset,
        parameters=plasma_parameters,
        timerange=timerange,
    ).retrieve_data()

    # get column names
    bcols = mag_vars.data[0].columns
    density_col = plasma_vars.data[0].columns[0]
    vec_cols = plasma_vars.data[1].columns
    temperature_col = plasma_vars.data[2].columns[0]

    # get data
    mag_data = mag_vars.to_polars()
    plasma_data = (
        plasma_vars.to_polars()
        .with_columns(plasma_speed=pl_norm(vec_cols))
        .rename({density_col: "plasma_density"})
    )
    # process temperature data
    if plasma_vars.data[2].unit == "km/s":
        plasma_data = plasma_data.pipe(df_thermal_spd2temp, temperature_col)
    else:
        plasma_data = plasma_data.rename({temperature_col: "plasma_temperature"})

    return IDsDataset(
        mag_data=mag_data.pipe(resample, every=ts),
        plasma_data=plasma_data,
        tau=tau,
        ts=ts,
        bcols=bcols,
        vec_cols=vec_cols,
        density_col="plasma_density",
        speed_col="plasma_speed",
        temperature_col="plasma_temperature",
    ).find_events(return_best_fit=False).update_candidates_with_plasma_data()

In [10]:
ids : list[pl.DataFrame] = []
for _tr in tqdm(TimeRange(timerange).split(8)):
    
    _timerange = [_tr.start.value, _tr.end.value]
    _id = get_and_process_data(
        mag_dataset, mag_parameters, plasma_dataset, plasma_parameters, _timerange, tau, ts,
        provider = provider
    )
    ids.append(_id.events)

  0%|          | 0/8 [00:00<?, ?it/s]15-Feb-24 13:44:40: Non compliant ISTP file: No data variable found, this is suspicious
15-Feb-24 13:44:40: Non compliant ISTP file: No data variable found, this is suspicious
15-Feb-24 13:44:40: Non compliant ISTP file: No data variable found, this is suspicious
15-Feb-24 13:44:40: Non compliant ISTP file: No data variable found, this is suspicious
15-Feb-24 13:44:40: Non compliant ISTP file: No data variable found, this is suspicious
15-Feb-24 13:44:40: Non compliant ISTP file: No data variable found, this is suspicious
15-Feb-24 13:44:40: Non compliant ISTP file: No data variable found, this is suspicious
15-Feb-24 13:44:40: Non compliant ISTP file: No data variable found, this is suspicious
15-Feb-24 13:44:40: Non compliant ISTP file: No data variable found, this is suspicious
15-Feb-24 13:44:40: Non compliant ISTP file: No data variable found, this is suspicious
15-Feb-24 13:44:40: Non compliant ISTP file: No data variable found, this is suspic

AttributeError: 'NoneType' object has no attribute 'columns'

## Solar wind pipeline

In [ ]:
#| export
from discontinuitypy.utils.basic import filter_tranges_df

def filter_sw_events(events: pl.LazyFrame, sw_state: pl.LazyFrame) -> pl.LazyFrame:
    
    start, end = sw_state.select(['start', 'end']).collect()
    sw_events = filter_tranges_df(events.collect(), (start, end))
    
    return sw_events

def create_sw_events_pipeline(
    sat_id,
    tau: int = 60,
    ts_mag: int = 1,
    
):
  
    ts_mag_str = f"ts_{ts_mag}s"
    tau_str = f"tau_{tau}s"
    
    node_filter_sw_events = node(
        filter_sw_events,
        inputs=[
            f"events.{sat_id}_{ts_mag_str}_{tau_str}",
            f"{sat_id}.STATE.inter_data_sw",
        ],
        outputs=f"events.{sat_id}_sw_{ts_mag_str}_{tau_str}"
        
    )

    nodes = [node_filter_sw_events]
    return pipeline(nodes)

## Pipelines

In [ ]:
# | export
def create_pipeline(
    sat_id="THB",
    params: Optional[dict] = None,
):
    
    if params is None:
        params = PARAMS
    tau = params["tau"]
    ts_state = params[sat_id]["STATE"].get("time_resolution", 0)
    ts_mag = params[sat_id]["MAG"].get("time_resolution", 0)
    ts_state_str = f"ts_{ts_state}s"


    input_combined_data = {
        f"{sat_id}.STATE.primary_data_{ts_state_str}": f"OMNI.LowRes.primary_data_{ts_state_str}"
    }
    
    node_combined_data = pipeline(
        create_combined_data_pipeline(sat_id),
        inputs=input_combined_data,
    )

    return (
        create_mag_data_pipeline(sat_id)
        + create_state_data_pipeline(sat_id)
        + node_combined_data
        + create_sw_events_pipeline(sat_id, tau=tau, ts_mag=ts_mag)
    )